# Техническое задание 2

**Тестирование автокорректоров, использующих контекстную корректировку слов.**

Дополнить предыдущую работу: найти еще 3-4 наиболее популярных автокорректора, которые используют контекстную корректировку слов. Проверить, как они работают.

## План работы

1. Найти библиотеки, которые используют контекстную корректировку слов.

2. Подготовить и загрузить уже распознанные и эталонные файлы из задания 1.

3. Автокоррекция текстов.

  3.1. *Применение автокорректора:* Взять распознанный текст и применить к нему выбранные модели автокорректоров. Сохранить исправленный текст в отдельную переменную.

  3.2. *Оценка результатов:* Сравнить эталонный текст с исправленным текстом. Оценить количество оставшихся ошибок после применения автокорректора, используя метрики WER и CER.

4. Анализ и выводы
Провести сравнительный анализ полученных результатов, включая автокорректоры из Задания 1 и сделать выводы о качестве работы автокорректоров.

In [1]:
# Удаляем метаданные виджетов
from IPython.display import display, Javascript

display(Javascript('''
    const nb = IPython.notebook;
    if (nb.metadata.widgets) {
        delete nb.metadata.widgets;
        nb.save();
    }
'''))

<IPython.core.display.Javascript object>

## **1. Библиотеки для контекстной коррекции**

Список библиотек контекстной коррекции

| Библиотека    | Тип           | Описание |
| ------------- | ------------- |----------|
| JamSpell  | 🔹 ML-модель  |Контекстная коррекция на основе языковой модели, работает быстро|
| SymSpell  | 🔹 Алгоритм  |Поддерживает контекстные исправления, но слабее, чем нейросети|
| Hunspell |🔹 Правила | Используется в LibreOffice, проверяет морфологию и контекст|
| LanguageTool|🔹 ML + Правила|Очень мощный, исправляет грамматику и орфографию (есть API)
| DeepPavlov|🔹 Нейросеть|Основан на трансформерах, хорошо понимает контекст|
| Aspell|🔹 Алгоритм|Поддерживает контекстные исправления, но ограничен словарями|
| BERTSpell|🔹 Нейросеть (BERT)|Использует BERT для исправления ошибок в контексте (state-of-the-art)|
| GECToR |🔹 Нейросеть (T5, RoBERTa)|Лучшая библиотека для грамматической и орфографической коррекции

Выберем из них 5 моделей: JamSpell, Aspell, LanguageTool, RU T5, RuM2M100-1.2B

## **2. Загрузка файлов для исследования**

**Установка библиотек**

In [ ]:
pip install pytesseract pillow jiwer # установка библиотеки для вычисления метрик

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pytesseract
import os
import requests
import subprocess
import re
import string
import jiwer # библиотека для вычисления метрик
from PIL import Image, ImageFilter
from io import BytesIO

Для JamSpell

In [ ]:
!!sudo apt-get install swig3.0 # для JamSpell

['',
 'Reading package lists... 0%',
 '',
 'Reading package lists... 0%',
 '',
 'Reading package lists... 0%',
 '',
 'Reading package lists... 3%',
 '',
 'Reading package lists... 3%',
 '',
 'Reading package lists... 4%',
 '',
 'Reading package lists... 4%',
 '',
 'Reading package lists... 28%',
 '',
 'Reading package lists... 38%',
 '',
 'Reading package lists... 38%',
 '',
 'Reading package lists... 38%',
 '',
 'Reading package lists... 38%',
 '',
 'Reading package lists... 45%',
 '',
 'Reading package lists... 45%',
 '',
 'Reading package lists... 55%',
 '',
 'Reading package lists... 55%',
 '',
 'Reading package lists... 56%',
 '',
 'Reading package lists... 58%',
 '',
 'Reading package lists... 58%',
 '',
 'Reading package lists... 58%',
 '',
 'Reading package lists... 58%',
 '',
 'Reading package lists... 59%',
 '',
 'Reading package lists... 59%',
 '',
 'Reading package lists... 59%',
 '',
 'Reading package lists... 59%',
 '',
 'Reading package lists... 65%',
 '',
 'Reading pack

In [ ]:
!sudo pip install jamspell

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for jamspell: filename=jamspell-0.0.12-cp311-cp311-linux_x86_64.whl size=1785475 sha256=05d3205bbb8f8908cc7df0f9c6185796ff64cbbf287e9389fef1d6c99eac9e42
  Stored in directory: /root/.cache/pip/wheels/2c/f7/b1/1314c3a8f44a8e2bbe9f90b2ae05c404068f797b7681e1163b
Successfully built jamspell


In [ ]:
!wget -O ru.bin https://github.com/bakwc/JamSpell-models/raw/master/ru.tar.gz
!tar -xzf ru.bin || echo "Ошибка: файл ru.bin не распакован"

--2025-02-09 17:11:45--  https://github.com/bakwc/JamSpell-models/raw/master/ru.tar.gz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bakwc/JamSpell-models/master/ru.tar.gz [following]
--2025-02-09 17:11:46--  https://raw.githubusercontent.com/bakwc/JamSpell-models/master/ru.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39663667 (38M) [application/octet-stream]
Saving to: ‘ru.bin’

ru.bin              100%[===================>]  37.83M   116MB/s    in 0.3s    

2025-02-09 17:11:48 (116 MB/s) - ‘ru.bin’ saved [39663667/39663667]



In [ ]:
import jamspell

Для Aspell

In [ ]:
!sudo apt-get install aspell aspell-ru -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  dictionaries-common libaspell15 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist
The following NEW packages will be installed:
  aspell aspell-ru dictionaries-common libaspell15 libtext-iconv-perl
0 upgraded, 5 newly installed, 0 to remove and 19 not upgraded.
Need to get 947 kB of archives.
After this operation, 3,799 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libtext-iconv-perl amd64 1.7-7build3 [14.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libaspell15 amd64 0.60.8-4build1 [325 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 dictionaries-common all 1.28.14 [185 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 aspell amd64 0.60.8-4build1 [87.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aspell-ru 

In [ ]:
import subprocess

Для LanguageTool

In [ ]:
pip install language-tool-python

In [ ]:
from language_tool_python import LanguageTool

Для RU T5

In [ ]:
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 748.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install nltk

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize, regexp_tokenize, word_tokenize

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Для M2M100

In [ ]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

**Литературный текст**

In [ ]:
from google.colab import drive # подключение гугл диска с файлами
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
rec_lit_text = '/content/drive/My Drive/Files_for_project/rec_lit_text.txt'
rec_lit_text = open(rec_lit_text, 'r', encoding='utf-8').read()

In [ ]:
etalon_lit_text = '/content/drive/My Drive/Files_for_project/etalon_lit_text.txt'
etalon_lit_text = open(etalon_lit_text, 'r', encoding='utf-8').read()

In [ ]:
print(rec_lit_text[0:500])
print(etalon_lit_text[0:500])

по-видимому, с вестибюля. В заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода, в нишах между колоннами в неестественных позах торчали статуи в ржавых латах, справа от входа у стены громоздились обломки древних идолов, наверху этой кучи торчали гипсовые ноги в сапогах. С почерневших портретов под потолком строго взирали маститые старцы, в их лицах усматривались знакомые черты Федора Симеоновича, товарища Жиана Жиакомо и других мастеров. Весь этот архаический 
﻿по-видимому, с вестибюля. В заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода, в нишах между колоннами в неестественных позах торчали статуи в ржавых латах, справа от входа у стены громоздились обломки древних идолов, наверху этой кучи торчали гипсовые ноги в сапогах. С почерневших портретов под потолком строго взирали маститые старцы, в их лицах усматривались знакомые черты Федора Симеоновича, товарища Жиана Жиакомо и других мастеров. Весь этот архаически

**Учебный текст**

In [ ]:
rec_educ_text = '/content/drive/My Drive/Files_for_project/rec_educ_text.txt'
rec_educ_text = open(rec_educ_text, 'r', encoding='utf-8').read()

In [ ]:
etalon_educ_text = '/content/drive/My Drive/Files_for_project/etalon_educ_text.txt'
etalon_educ_text = open(etalon_educ_text, 'r', encoding='utf-8').read()

In [ ]:
print(rec_educ_text[0:500])
print(etalon_educ_text[0:500])

Субъектом правопонимания всегда выступает конкретный человек, например: а) гражданин, обладающий минимальным правовым кругозором, столкнувшийся с проблемами права вообще; 6) юрист-профессионал, имеющий достаточный запас знаний о праве, способный применять и толковать правовые нормы; в) ученый, человек с абстрактным мышлением, занимающийся изучением права, обладающий суммой исторических и современных знаний, способный к интерпретации не только норм, но и принципов права, владеющий определенной ме
﻿Субъектом правопонимания всегда выступает конкретный человек, например: а) гражданин, обладающий минимальным правовым кругозором, столкнувшийся с проблемами права вообще; б) юрист-профессионал, имеющий достаточный запас знаний о праве, способный применять и толковать правовые нормы; в) ученый, человек с абстрактным мышлением, занимающийся изучением права, обладающий суммой исторических и современных знаний, способный к интерпретации не только норм, но и принципов права, владеющий определенной 

**Функции для вычисления метрик**

In [ ]:
def preprocess_text(text):
    """
    Функция для обработки текста для CER
    Приводит текст к нижнему регистру и удаляет знаки препинания и пробелы.
    """
    return ''.join(c.lower() for c in text if c not in string.punctuation and not c.isspace())

In [ ]:
def metrics_wer_cer(etalon_text, recognized_text):
    """
    Вычисление метрик CER и WER
    """
    # Обработка текстов для CER
    etalon_text_prep = preprocess_text(etalon_text)
    recognized_text_prep = preprocess_text(recognized_text)

    # Вычисление CER и WER
    wer = jiwer.wer(etalon_text, recognized_text)
    cer = jiwer.cer(etalon_text_prep, recognized_text_prep)

    return round(wer, 3), round(cer, 3)

## **3. Автокоррекция текстов**

### JamSpell

JamSpell — это библиотека для контекстной проверки орфографии, которая использует модели на основе n-грамм. Она поддерживает русский язык и другие языки.

Особенности:
Использует контекст для исправления ошибок.

Поддерживает предобученные модели для русского языка.

Быстрая и эффективная.

In [ ]:
# Загружаем автокорректор
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel("ru_small.bin")  # Подключаем предобученную модель

True

**Литературный текст**

In [ ]:
# Автокоррекция литературного текста
rec_lit_text_corr = corrector.FixFragment(rec_lit_text)

print("Оригинальный текст:", rec_lit_text[0:1000])
print("Исправленный текст:", rec_lit_text_corr[0:1000])

Оригинальный текст: по-видимому, с вестибюля. В заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода, в нишах между колоннами в неестественных позах торчали статуи в ржавых латах, справа от входа у стены громоздились обломки древних идолов, наверху этой кучи торчали гипсовые ноги в сапогах. С почерневших портретов под потолком строго взирали маститые старцы, в их лицах усматривались знакомые черты Федора Симеоновича, товарища Жиана Жиакомо и других мастеров. Весь этот архаический хлам надлежало давным-давно выбросить, прорубить в стенах окна и установить трубки дневного света, но все было заприходовано, заинвентаризовано и лично Модестом Матвеевичем к разбазариванию запрещено.  На капителях колонн и в лабиринтах исполинской люстры, свисающей с почерневшего потолка, шуршали нетопыри и летучие собаки. С ними Модест Матвеевич боролся. Он поливал их скипидаром и креозотом, опылял дустом, опрыскивал гексахлораном, они гибли тысячами, но возрождались десятк

In [ ]:
# проверим кандитатов на замену в 8 слове "мерцали"
corrector.GetCandidates(rec_lit_text.split(), 7)

('медали',
 'умерли',
 'держали',
 'морали',
 'мерами',
 'мешали',
 'мечтали',
 'мерапи',
 'вымерли',
 'умирали',
 'мерлин',
 'минерал',
 'мерцали',
 'мерлину',
 'мерлина')

Несмотря на то, что в словаре было слово "мерцали", алгоритм заменил его на "медали", так как модель учитывает контекст и фраза "мерцали кости" кажется модели менее вероятной, чем "медали кости". Посмотрим, какие слова были заменены.

In [ ]:
def get_corrections(original_text):
    corrected_text = corrector.FixFragment(original_text)

    # Сравниваем оригинальный и исправленный текст
    original_words = original_text.split()
    corrected_words = corrected_text.split()

    # Находим исправления
    corrections = []
    for orig_word, corr_word in zip(original_words, corrected_words):
        if orig_word != corr_word:
            # Сохраняем пару (исходное, исправленное)
            corrections.append((orig_word, corr_word))

    return corrected_text, corrections

# Автокоррекция и поиск исправлений
rec_lit_text_corr, corrections = get_corrections(rec_lit_text)

# Вывод результатов
print("🔍 Исправления:")
for orig, corr in corrections:
    print(f"- {orig} → {corr}")

🔍 Исправления:
- мерцали → медали
- мерно → зерно
- капала → канала
- почерневших → потерпевших
- усматривались → устраивались
- Жиана → Жана
- Жиакомо → Ираком
- Матвеевичем → Матвеевич
- свисающей → свисающий
- почерневшего → потерпевшего
- шуршали → украли
- опылял → отплыл
- дустом, → дуэтом,
- паиболее → наиболее
- нище, → ниже,
- застонал → затронул
- загремел → загребе
- спаружи → снаружи
- простирался → пробирался
- Ойре-Ойре, → Одре-Одре,
- изуверов, → призеров,
- шипя → шипы
- кривясь → кривая
- варил → верил
- оптнмизм. → оптимизм.
- разрушающих → разрушающие
- сублимировали → субсидировали
- отомкнул → толкнул
- полюбовался, → пользовался,
- Граафа. → Графа.
- рубильника → рубильник
- попятился → появился
- Обесточивать → Обеспечивать
- еженощно → ежегодно
- домовик → домов
- сосланный → посланный
- Вием → Всем
- вниварии → внимании
- вивария, → викария,
- потупился. → поступил.
- ‘сипло → ‘сияло
- вивария → викария
- Лернейской → Еврейской
- неразговорчивой → разговорчивой

In [ ]:
# Метрики литературного текста
wer_lit_jamspell, cer_lit_jamspell = metrics_wer_cer(etalon_lit_text,
                                                     rec_lit_text_corr)
print(f"WER: {wer_lit_jamspell}")
print(f"CER: {cer_lit_jamspell}")

WER: 0.08
CER: 0.015


**Текст из учебника**

In [ ]:
# Автокоррекция учебного текста
rec_educ_text_corr = corrector.FixFragment(rec_educ_text)

print("Оригинальный текст:", rec_educ_text[0:2000])
print("Исправленный текст:", rec_educ_text_corr[0:2000])

Оригинальный текст: Субъектом правопонимания всегда выступает конкретный человек, например: а) гражданин, обладающий минимальным правовым кругозором, столкнувшийся с проблемами права вообще; 6) юрист-профессионал, имеющий достаточный запас знаний о праве, способный применять и толковать правовые нормы; в) ученый, человек с абстрактным мышлением, занимающийся изучением права, обладающий суммой исторических и современных знаний, способный к интерпретации не только норм, но и принципов права, владеющий определенной методологией исследования. Правопонимание всегда субъективно, оригинально, хотя представления о праве могут совпадать у группы лиц и у целых слоев, классов.  Объектом правопонимания могут быть право в планетарном масштабе, право конкретного общества, отрасль, институт права, отдельные правовые нормы. При этом знания об отдельных структурных элементах экстраполируются на право в целом. Важную познавательную нагрузку здесь несут среда и взаимодействующие с правом общественные явл

In [ ]:
# Метрики учебного текста
wer_educ_jamspell, cer_educ_jamspell = metrics_wer_cer(etalon_educ_text,
                                                       rec_educ_text_corr)
print(f"WER: {wer_educ_jamspell}")
print(f"CER: {cer_educ_jamspell}")

WER: 0.028
CER: 0.003


**Результат:**

Автокорректор работает быстро. Заменяет иногда исходя их своего понимания контекста, не попадая в настоящий контекст. Таким образом, получаются исправления типа: "бродила → родила", "увязались → связались". Кажется, что понимание контекста литературного текста очень не просто передать программе.

### Aspell

In [ ]:
def correct_text_aspell(text):
    """Функция для автокоррекции"""
    process = subprocess.Popen(
        ["aspell", "-a", "--lang=ru"],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        text=True
    )

    # Передаём текст в Aspell
    stdout, _ = process.communicate(text)

    # Разбираем вывод Aspell
    lines = stdout.split("\n")[1:]  # Пропускаем первую строку Aspell
    words = text.split() # Разбиваем текст на слова

    corrected_text = []
    index = 0
    for line in lines:
        if line.startswith("&"):
            suggestions = line.split(": ")[1].split(", ")
            corrected_text.append(suggestions[0] if suggestions else words[index])
        else:
            if index < len(words):
                corrected_text.append(words[index])
        index += 1

    return " ".join(corrected_text)

**Литературный текст**

In [ ]:
# Автокоррекция литературного текста
lit_text_aspell_corr = correct_text_aspell(rec_lit_text)

print("Оригинальный текст:", rec_lit_text[0:1000])
print("Исправленный текст:", lit_text_aspell_corr[0:1000])

Оригинальный текст: по-видимому, с вестибюля. В заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода, в нишах между колоннами в неестественных позах торчали статуи в ржавых латах, справа от входа у стены громоздились обломки древних идолов, наверху этой кучи торчали гипсовые ноги в сапогах. С почерневших портретов под потолком строго взирали маститые старцы, в их лицах усматривались знакомые черты Федора Симеоновича, товарища Жиана Жиакомо и других мастеров. Весь этот архаический хлам надлежало давным-давно выбросить, прорубить в стенах окна и установить трубки дневного света, но все было заприходовано, заинвентаризовано и лично Модестом Матвеевичем к разбазариванию запрещено.  На капителях колонн и в лабиринтах исполинской люстры, свисающей с почерневшего потолка, шуршали нетопыри и летучие собаки. С ними Модест Матвеевич боролся. Он поливал их скипидаром и креозотом, опылял дустом, опрыскивал гексахлораном, они гибли тысячами, но возрождались десятк

In [ ]:
# Метрики литературного текста
wer_lit_aspell, cer_lit_aspell = metrics_wer_cer(etalon_lit_text,
                                                 lit_text_aspell_corr)
print(f"WER: {wer_lit_aspell}")
print(f"CER: {cer_lit_aspell}")

WER: 0.108
CER: 0.081


**Текст из учебника**

In [ ]:
# Автокоррекция учебного текста
educ_text_aspell_corr = correct_text_aspell(rec_educ_text)

print("Оригинальный текст:", rec_educ_text[0:2000])
print("Исправленный текст:", educ_text_aspell_corr[0:2000])

Оригинальный текст: Субъектом правопонимания всегда выступает конкретный человек, например: а) гражданин, обладающий минимальным правовым кругозором, столкнувшийся с проблемами права вообще; 6) юрист-профессионал, имеющий достаточный запас знаний о праве, способный применять и толковать правовые нормы; в) ученый, человек с абстрактным мышлением, занимающийся изучением права, обладающий суммой исторических и современных знаний, способный к интерпретации не только норм, но и принципов права, владеющий определенной методологией исследования. Правопонимание всегда субъективно, оригинально, хотя представления о праве могут совпадать у группы лиц и у целых слоев, классов.  Объектом правопонимания могут быть право в планетарном масштабе, право конкретного общества, отрасль, институт права, отдельные правовые нормы. При этом знания об отдельных структурных элементах экстраполируются на право в целом. Важную познавательную нагрузку здесь несут среда и взаимодействующие с правом общественные явл

In [ ]:
# Метрики учебного текста
wer_educ_aspell, cer_educ_aspell = metrics_wer_cer(etalon_educ_text,
                                                   educ_text_aspell_corr)
print(f"WER: {wer_educ_aspell}")
print(f"CER: {cer_educ_aspell}")

WER: 0.094
CER: 0.066


**Результат:**

Метрики ухудшились по срванению с метриками распознанного текста без коррекции. Заменяет довольно распространенные слова на другие, "где-то" на "гетто", "давным-давно" на "Давыдовною". Имена собственные заменяет на более распространенные.

### LanguageTool

In [ ]:
tool = LanguageTool('ru-RU')  # Указываем язык (русский)

INFO:language_tool_python.download_lt:Unzipping /tmp/tmpu5phgf8f.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /root/.cache/language_tool_python.


In [ ]:
# функция для автокоррекции
def correct_with_languagetool(text):
    matches = tool.check(text)
    corrected_text = tool.correct(text)

    # Список для хранения исправлений
    corrections = []

    for match in matches:
        if match.replacements:  # Проверяем, есть ли предложения для замены
            original = text[match.offset:match.offset + match.errorLength]
            # Берем первое предложенное исправление
            corrected = match.replacements[0]
            corrections.append((original, corrected))

    return corrected_text, corrections

**Литературный текст**

In [ ]:
# Исправление текста
corr_lit_text_lt, corrections_lit = correct_with_languagetool(rec_lit_text)

# Вывод исправленного текста
print("Оригинальный текст:", rec_lit_text[0:1000])
print("Исправленный текст:", corr_lit_text_lt[0:1000])

Оригинальный текст: по-видимому, с вестибюля. В заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода, в нишах между колоннами в неестественных позах торчали статуи в ржавых латах, справа от входа у стены громоздились обломки древних идолов, наверху этой кучи торчали гипсовые ноги в сапогах. С почерневших портретов под потолком строго взирали маститые старцы, в их лицах усматривались знакомые черты Федора Симеоновича, товарища Жиана Жиакомо и других мастеров. Весь этот архаический хлам надлежало давным-давно выбросить, прорубить в стенах окна и установить трубки дневного света, но все было заприходовано, заинвентаризовано и лично Модестом Матвеевичем к разбазариванию запрещено.  На капителях колонн и в лабиринтах исполинской люстры, свисающей с почерневшего потолка, шуршали нетопыри и летучие собаки. С ними Модест Матвеевич боролся. Он поливал их скипидаром и креозотом, опылял дустом, опрыскивал гексахлораном, они гибли тысячами, но возрождались десятк

In [ ]:
# Вывод исправленных слов
print("\n🔍 Найденные исправления:")
for original, corrected in corrections_lit:
    print(f"❌ {original} → {corrected}")


🔍 Найденные исправления:
❌ по-видимому → По-видимому
❌ Жиана → Диана
❌ Жиакомо → Диаком
❌ паиболее → наиболее
❌ ктото → кто-то
❌ ,— строго → , — строго
❌ .— Что → . — Что
❌ спаружи → снаружи
❌ и тем не менее → тем не менее
❌    →  
❌    →  
❌ оптнмизм → оптимизм
❌ субмолекулярной → бимолекулярной
❌ экстракфты → экстракты
❌ гореутолителей → горе утолителей
❌ злободроби → глыбодробе
❌ тель → телль
❌ Вием → Аием
❌ в Соловец → в соловце
❌    →  
❌    →  
❌ Бидструпа → Пеструха
❌ вниварии → виварии
❌    →  
❌ Бриарей → Бри арей
❌ ,— сказал → , — сказал
❌ ,— и → , — и
❌ ,— сказал → , — сказал
❌ ,— сказал → , — сказал
❌ .— Пойдем → . — Пойдем
❌ в глубь → вглубь
❌ пятидесятиголовые → пятидесятидолларовые
❌ братцыблизнецы → братцы близнецы
❌ Гиес → Гияс
❌ Котт → Готт
❌ Бриарей → Бри арей
❌ .— Болит → . — Болит
❌ по-эллински → доэллински
❌ ,— сказала → , — сказала
❌ притихЛи → притихли
❌    →  
❌ Бриарей → Бри арей
❌ Ну-ну-ну → Ну-ну
❌ ,— сказал → , — сказал
❌ Бриарей → Бри арей
❌ Бриарей → 

Видно, что помимо ошибок автокорректор исправил некоторые имена собственные, фантазийные слова, пробел перед тире в прямой речи, сложные слова, которых нет у него в словаре. Пробовала игнорировать ",-", но метрики стали хуже.

In [ ]:
# Метрики литературного текста
wer_lit_lt, cer_lit_lt = metrics_wer_cer(etalon_lit_text,
                                         corr_lit_text_lt)
print(f"WER: {wer_lit_lt}")
print(f"CER: {cer_lit_lt}")

WER: 0.058
CER: 0.009


**Текст из учебника**

In [ ]:
# Исправление текста
corr_educ_text_lt, corrections_educ = correct_with_languagetool(rec_educ_text)

# Вывод исправленного текста
print("Оригинальный текст:", rec_educ_text[0:1000])
print("Исправленный текст:", corr_educ_text_lt[0:1000])

Оригинальный текст: Субъектом правопонимания всегда выступает конкретный человек, например: а) гражданин, обладающий минимальным правовым кругозором, столкнувшийся с проблемами права вообще; 6) юрист-профессионал, имеющий достаточный запас знаний о праве, способный применять и толковать правовые нормы; в) ученый, человек с абстрактным мышлением, занимающийся изучением права, обладающий суммой исторических и современных знаний, способный к интерпретации не только норм, но и принципов права, владеющий определенной методологией исследования. Правопонимание всегда субъективно, оригинально, хотя представления о праве могут совпадать у группы лиц и у целых слоев, классов.  Объектом правопонимания могут быть право в планетарном масштабе, право конкретного общества, отрасль, институт права, отдельные правовые нормы. При этом знания об отдельных структурных элементах экстраполируются на право в целом. Важную познавательную нагрузку здесь несут среда и взаимодействующие с правом общественные явл

In [ ]:
# Вывод исправленных слов
print("\n🔍 Найденные исправления:")
for original, corrected in corrections_educ:
    print(f"❌ {original} → {corrected}")


🔍 Найденные исправления:
❌ правопонимания → право понимания
❌ Правопонимание → Право понимание
❌ правопонимания → право понимания
❌ правопонимания → право понимания
❌ правопонимание → право понимание
❌ правопонимание → право понимание
❌ правопонимания → право понимания
❌ правопонимания → право понимания
❌    →  
❌ правопонимания → право понимания
❌ правообразования → кровообразования
❌ социальноценностный → социально ценностный
❌ Вл → Л
❌ неотомисты → диатомиты
❌    →  
❌    →  
❌ правообразования → кровообразования
❌ ,— государство → , — государство
❌ ,— подчеркивал → , — подчеркивал
❌ ,— стремится → , — стремится
❌ революций → Революций
❌ естественно-правовой → имущественно-правовой
❌ Бергбом → Бергом
❌ оппози → аппазе
❌ Цит → Жить
❌ по → По
❌    →  


In [ ]:
# Метрики учебного текста
wer_educ_lt, cer_educ_lt = metrics_wer_cer(etalon_educ_text,
                                           corr_educ_text_lt)
print(f"WER: {wer_educ_lt}")
print(f"CER: {cer_educ_lt}")

WER: 0.053
CER: 0.005


**Результат:**

Этот автокорректор кроме орфографических ошибок заменяет сложные слова, неправильную пунктуацию, имена собственные, фантазийные слова, придуманные автором.

### RU T5

rut5-base-multitask — это предобученная модель на основе архитектуры T5 (Text-To-Text Transfer Transformer), адаптированная для работы с русским языком. Она размещена на платформе Hugging Face.

T5 (Text-to-Text Transfer Transformer) — это модель от Google, обученная переводить любые задачи обработки текста

In [ ]:
# Загрузка модели и токенизатора
model_name = "cointegrated/rut5-base-multitask"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

In [ ]:
# Использование GPU, если доступно
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Функция для разбиения длинных предложений на части
def split_long_sentence(sentence, max_tokens=450, overlap=50):
    words = word_tokenize(sentence, language='russian')
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(tokenizer.encode(' '.join(current_chunk))) >= max_tokens:
            chunks.append(' '.join(current_chunk))
            # перекрытие для сохранения контекста
            current_chunk = current_chunk[-overlap:]

    if current_chunk:
        chunks.append(' '.join(current_chunk))
    return chunks

# Функция для исправления текста с учетом предложений
def correct_text_rut5(text):
    sentences = sent_tokenize(text, language='russian')
    corrected_sentences = []

    for sentence in sentences:
        # Проверка длины предложения
        if len(tokenizer.encode(sentence)) > 512:
            parts = split_long_sentence(sentence)
        else:
            parts = [sentence]

        corrected_parts = []
        for part in parts:
            input_text = f"Исправь ошибки, не изменяя стиль: {part}"
            inputs = tokenizer.encode(input_text,
                                      return_tensors="pt",
                                      max_length=512,
                                      truncation=True).to(device)

            outputs = model.generate(
                inputs,
                max_length=512,
                num_beams=4,
                do_sample=False,
                repetition_penalty=1.0
            )

            corrected_part = tokenizer.decode(
                outputs[0],
                skip_special_tokens=True
                )
            corrected_parts.append(corrected_part)

        # Объединение исправленных частей текста
        merged_sentence = ' '.join(corrected_parts)
        corrected_sentences.append(merged_sentence)

    return ' '.join(corrected_sentences)

# Пример длинного текста с ошибками
text_with_errors = """
В заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода,
в нишах между колоннами в неестественных позах торчали статуи в ржавых латах,
справа от входа у стены громоздились обломки древних идолов, наверху этой кучи торчали гипсовые ноги в сапогах.
"""

# Исправление текста
corrected_text = correct_text_rut5(text_with_errors)

# Результат
print("🔹 Оригинальный текст:")
print(text_with_errors)
print("\n✅ Исправленный текст:")
print(corrected_text)


🔹 Оригинальный текст:

В заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода,
в нишах между колоннами в неестественных позах торчали статуи в ржавых латах,
справа от входа у стены громоздились обломки древних идолов, наверху этой кучи торчали гипсовые ноги в сапогах.


✅ Исправленный текст:
В заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода, в нишах между колоннами в неестественных позах торчали статуи в ржавых латах.


**Литературный текст**

In [ ]:
corr_lit_text_rut5 = correct_text_rut5(rec_lit_text)

In [ ]:
print("Оригинальный текст:", rec_lit_text[0:2500])
print("Исправленный текст:", corr_lit_text_rut5[0:2500])

Оригинальный текст: по-видимому, с вестибюля. В заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода, в нишах между колоннами в неестественных позах торчали статуи в ржавых латах, справа от входа у стены громоздились обломки древних идолов, наверху этой кучи торчали гипсовые ноги в сапогах. С почерневших портретов под потолком строго взирали маститые старцы, в их лицах усматривались знакомые черты Федора Симеоновича, товарища Жиана Жиакомо и других мастеров. Весь этот архаический хлам надлежало давным-давно выбросить, прорубить в стенах окна и установить трубки дневного света, но все было заприходовано, заинвентаризовано и лично Модестом Матвеевичем к разбазариванию запрещено.  На капителях колонн и в лабиринтах исполинской люстры, свисающей с почерневшего потолка, шуршали нетопыри и летучие собаки. С ними Модест Матвеевич боролся. Он поливал их скипидаром и креозотом, опылял дустом, опрыскивал гексахлораном, они гибли тысячами, но возрождались десятк

In [ ]:
# Метрики литературного текста
wer_lit_rut5, cer_lit_rut5 = metrics_wer_cer(etalon_lit_text,
                                             corr_lit_text_rut5)
print(f"WER: {wer_lit_rut5}")
print(f"CER: {cer_lit_rut5}")

WER: 0.491
CER: 0.314


**Текст из учебника**

In [ ]:
corr_educ_text_rut5 = correct_text_rut5(rec_educ_text)
print("Оригинальный текст:", rec_educ_text[0:1000])
print("Исправленный текст:", corr_educ_text_rut5[0:1000])

Оригинальный текст: Субъектом правопонимания всегда выступает конкретный человек, например: а) гражданин, обладающий минимальным правовым кругозором, столкнувшийся с проблемами права вообще; 6) юрист-профессионал, имеющий достаточный запас знаний о праве, способный применять и толковать правовые нормы; в) ученый, человек с абстрактным мышлением, занимающийся изучением права, обладающий суммой исторических и современных знаний, способный к интерпретации не только норм, но и принципов права, владеющий определенной методологией исследования. Правопонимание всегда субъективно, оригинально, хотя представления о праве могут совпадать у группы лиц и у целых слоев, классов.  Объектом правопонимания могут быть право в планетарном масштабе, право конкретного общества, отрасль, институт права, отдельные правовые нормы. При этом знания об отдельных структурных элементах экстраполируются на право в целом. Важную познавательную нагрузку здесь несут среда и взаимодействующие с правом общественные явл

In [ ]:
# Метрики учебного текста
wer_educ_rut5, cer_educ_rut5 = metrics_wer_cer(etalon_educ_text,
                                               corr_educ_text_rut5)
print(f"WER: {wer_educ_rut5}")
print(f"CER: {cer_educ_rut5}")

WER: 0.315
CER: 0.221


**Попытка улучшения**

Так как модель обрезает входящее предложение до 512 токенов, мне пришлось разбить предложение по запятым и подать на вход модели эти небольшие кусочки.



In [ ]:
# Функция для исправления текста
def correct_text_rut5(text):
    # Разделение текста на предложения с помощью регулярных выражений
    sentences = regexp_tokenize(text, pattern=r'[^!?\.,]+[!?\.,]?', gaps=False)
    corrected_sentences = []

    for sentence in sentences:
        input_text = f"grammar: {sentence.strip()}"  # Указываем задачу модели
        inputs = tokenizer.encode(input_text,
                                  return_tensors="pt",
                                  max_length=512,
                                  truncation=True).to(device)

        # Генерация исправленного текста
        outputs = model.generate(
            inputs,
            max_length=512,
            num_beams=1,        # Повышение качества за счёт лучевого поиска
            do_sample=False,     # Вариативность исправлений
            repetition_penalty=1.0
        )

        corrected_sentence = tokenizer.decode(outputs[0],
                                              skip_special_tokens=True)
        corrected_sentences.append(corrected_sentence)

    corrected_text = ' '.join(corrected_sentences)
    return corrected_text

In [ ]:
# Пример длинного текста с ошибками
text_with_errors = """
по-видимому, с вестибюля. В заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода,
в нишах между колоннами в неестественных позах торчали статуи в ржавых латах,
справа от входа у стены громоздились обломки древних идолов, наверху этой кучи торчали гипсовые ноги в сапогах.
"""

# Исправление текста
corrected_text = correct_text_rut5(text_with_errors)

# Результат
print("🔹 Оригинальный текст:")
print(text_with_errors)
print("\n✅ Исправленный текст:")
print(corrected_text)


🔹 Оригинальный текст:

по-видимому, с вестибюля. В заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода,
в нишах между колоннами в неестественных позах торчали статуи в ржавых латах,
справа от входа у стены громоздились обломки древних идолов, наверху этой кучи торчали гипсовые ноги в сапогах.


✅ Исправленный текст:
По-видимому, я знаю, что я знаю. С вестибюля. В заплесневелых углах белесо мерцали кости прикованных скелетов. где-то мерно капала вода. В нишах между колоннами торчали статуи в ржавых латах. справа от входа у стены громоздились обломки древних идолов. Наверх этой кучи торчали гипсовые ноги в сапогах. .


**Литературный текст**

In [ ]:
corr_lit_text_rut5 = correct_text_rut5(rec_lit_text)
print("Оригинальный текст:", rec_lit_text[0:1000])
print("Исправленный текст:", corr_lit_text_rut5[0:1000])

Оригинальный текст: по-видимому, с вестибюля. В заплесневелых углах белесо мерцали кости прикованных скелетов, где-то мерно капала вода, в нишах между колоннами в неестественных позах торчали статуи в ржавых латах, справа от входа у стены громоздились обломки древних идолов, наверху этой кучи торчали гипсовые ноги в сапогах. С почерневших портретов под потолком строго взирали маститые старцы, в их лицах усматривались знакомые черты Федора Симеоновича, товарища Жиана Жиакомо и других мастеров. Весь этот архаический хлам надлежало давным-давно выбросить, прорубить в стенах окна и установить трубки дневного света, но все было заприходовано, заинвентаризовано и лично Модестом Матвеевичем к разбазариванию запрещено.  На капителях колонн и в лабиринтах исполинской люстры, свисающей с почерневшего потолка, шуршали нетопыри и летучие собаки. С ними Модест Матвеевич боролся. Он поливал их скипидаром и креозотом, опылял дустом, опрыскивал гексахлораном, они гибли тысячами, но возрождались десятк

In [ ]:
# Метрики литературного текста
wer_lit_rut5_corr, cer_lit_rut5_corr = metrics_wer_cer(etalon_lit_text,
                                                       corr_lit_text_rut5)
print(f"WER: {wer_lit_rut5_corr}")
print(f"CER: {cer_lit_rut5_corr}")

WER: 0.657
CER: 0.385


**Текст из учебника**

In [ ]:
corr_educ_text_rut5 = correct_text_rut5(rec_educ_text)
print("Оригинальный текст:", rec_educ_text[0:1000])
print("Исправленный текст:", corr_educ_text_rut5[0:1000])

Оригинальный текст: Субъектом правопонимания всегда выступает конкретный человек, например: а) гражданин, обладающий минимальным правовым кругозором, столкнувшийся с проблемами права вообще; 6) юрист-профессионал, имеющий достаточный запас знаний о праве, способный применять и толковать правовые нормы; в) ученый, человек с абстрактным мышлением, занимающийся изучением права, обладающий суммой исторических и современных знаний, способный к интерпретации не только норм, но и принципов права, владеющий определенной методологией исследования. Правопонимание всегда субъективно, оригинально, хотя представления о праве могут совпадать у группы лиц и у целых слоев, классов.  Объектом правопонимания могут быть право в планетарном масштабе, право конкретного общества, отрасль, институт права, отдельные правовые нормы. При этом знания об отдельных структурных элементах экстраполируются на право в целом. Важную познавательную нагрузку здесь несут среда и взаимодействующие с правом общественные явл

In [ ]:
# Метрики учебного текста
wer_educ_rut5_corr, cer_educ_rut5_corr = metrics_wer_cer(etalon_educ_text,
                                                         corr_educ_text_rut5)
print(f"WER: {wer_educ_rut5_corr}")
print(f"CER: {cer_educ_rut5_corr}")

WER: 1.014
CER: 0.61


**Результат:**

Главная проблема модели - она принимает только 24 слова на вход (512 токенов). Проблема решается только тем, что можно делить предложения не по точкам, а по запятым и подавать на вход маленькие кусочки предложений. Однако, в этом случае, модель чувствует, что это странные предложения и начинает генерировать дополнительный текст и тем самым совсем меняет предложения. В результате такой обработки метрики стали еще хуже, а в тексте из учебника модель так нагенерировала новый текст, что метрика WER вообще не узнала текст WER 101%!

### Модель RuM2M100-1.2B

RuM2M100-1.2B — это многоязычная модель для машинного перевода и генерации текста, основанная на архитектуре M2M100 от Meta (ранее Facebook AI). Используется, в том числе, для контекстной коррекции текста.

Модель для проверки автокоррекции взята здесь: https://huggingface.co/ai-forever/RuM2M100-1.2B Модель исправляет орфографические ошибки и опечатки, приводя все слова в тексте к норме русского языка.

Эта модель работает медленно. С помощью счетчиков слов определила, что литературный текст содержит 3578 слов, учебный текст - 777 слов. Начнем работу с учебного текста, так как он меньше. Посчитаем количество токенов для обоих текстов.

Для увеличения скорости использован:
- Mixed Precision (FP16) — уменьшает размер весов в 2 раза, ускоряет вычисления
- device_map="auto" — автоматически размещает модель на GPU
- .to("cuda") — ускоряет обработку на видеокарте

In [ ]:
encodings = tokenizer(rec_lit_text)
print(f"Количество токенов: {len(encodings['input_ids'])}")

Количество токенов: 7982


In [ ]:
encodings = tokenizer(rec_educ_text)
print(f"Количество токенов: {len(encodings['input_ids'])}")

Количество токенов: 1615


In [ ]:
print(tokenizer.model_max_length)

1000000000000000019884624838656


In [ ]:
# Загружаем модель без device_map
path_to_model = "ai-forever/RuM2M100-1.2B"
model = M2M100ForConditionalGeneration.from_pretrained(
    path_to_model, torch_dtype=torch.float16
)

In [ ]:
# Перемещаем модель на GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Убедимся, что есть пунктуация для разделения
nltk.download('punkt_tab')
sentences = nltk.sent_tokenize(rec_educ_text, language="russian")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# Функция для обработки отдельных предложений
def process_text(text_chunk):
    encodings = tokenizer(text_chunk, return_tensors="pt")
    generated_tokens = model.generate(
        **encodings,
        max_length=512,          # Ограничиваем длину
        num_beams=3,             # Умеренное количество лучшей для скорости
        do_sample=False          # Случайная генерация
    )
    return tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    # Декодирование

**Текст из учебника**

In [ ]:
# Обработка текста по частям
results = []
for i, chunk in enumerate(sentences):
    try:
        result = process_text(chunk)
        results.append(result)
        print(f"✅ Обработано предложение {i+1}/{len(sentences)}")
    except Exception as e:
        print(f"❌ Ошибка в предложении {i+1}: {e}")

✅ Обработано предложение 1/37
✅ Обработано предложение 2/37
✅ Обработано предложение 3/37
✅ Обработано предложение 4/37
✅ Обработано предложение 5/37
✅ Обработано предложение 6/37
✅ Обработано предложение 7/37
✅ Обработано предложение 8/37
✅ Обработано предложение 9/37
✅ Обработано предложение 10/37
✅ Обработано предложение 11/37
✅ Обработано предложение 12/37
✅ Обработано предложение 13/37
✅ Обработано предложение 14/37
✅ Обработано предложение 15/37
✅ Обработано предложение 16/37
✅ Обработано предложение 17/37
✅ Обработано предложение 18/37
✅ Обработано предложение 19/37
✅ Обработано предложение 20/37
✅ Обработано предложение 21/37
✅ Обработано предложение 22/37
✅ Обработано предложение 23/37
✅ Обработано предложение 24/37
✅ Обработано предложение 25/37
✅ Обработано предложение 26/37
✅ Обработано предложение 27/37
✅ Обработано предложение 28/37
✅ Обработано предложение 29/37
✅ Обработано предложение 30/37
✅ Обработано предложение 31/37
✅ Обработано предложение 32/37
✅ Обработано пред

In [ ]:
# Собираем итоговый текст
educ_text_rum2m_corr = " ".join(results)
print(educ_text_rum2m_corr)

Субъектом правопонимания всегда выступает конкретный человек, например: а) гражданин, обладающий минимальным правовым кругозором, столкнувшийся с проблемами права вообще; 6) юрист-профессионал, имеющий достаточный запас знаний о праве, способный применять и толковать правовые нормы; в) ученый, человек с абстрактным мышлением, занимающийся изучением права, обладающий суммой исторических и современных знаний, способный к интерпретации не только норм, но и принципов права, владеющий определенной методологией исследования. Правопонимание всегда субъективно, оригинально, хотя представления о праве могут совпадать у группы лиц и у целых слоев, классов. Объектом правопонимания могут быть право в планетарном масштабе, право конкретного общества, отрасль, институт права, отдельные правовые нормы. При этом знания об отдельных структурных элементах экстраполируются на право в целом. Важную познавательную нагрузку здесь несут среда и взаимодействующие с правом общественные явления. Содержание прав

In [ ]:
# Метрики учебного текста
wer_educ_rum2m100, cer_educ_rum2m100 = metrics_wer_cer(etalon_educ_text,
                                                       educ_text_rum2m_corr)
print(f"WER: {wer_educ_rum2m100}")
print(f"CER: {cer_educ_rum2m100}")

WER: 0.019
CER: 0.002


**Результат:**

Метрики текста, обработанного моделью RuM2M100-1.2B показали себя очень хорошо на учебном тексте WER: 0.019 и CER: 0.002. Однако, скорость работы модели 44 минуты - 37 предложений или 1615 токенов. Это значит, что 7982 токенов будет обрабатываться примерно 4 часа. Принято решение не тестировать на литературном тексте. Однако, пока это лучшая метрика среди всех моделей!

## **4. Анализ и выводы**

In [ ]:
print("Таблица сравнения автокорректоров для литературного текста")
df = [
    [0.046,	0.004],
    [0.322,	0.055],
    [0.107,	0.027],
    [0.177,	0.005],
    [wer_lit_jamspell, cer_lit_jamspell],
    [wer_lit_aspell, cer_lit_aspell],
    [wer_lit_lt, cer_lit_lt],
    [wer_lit_rut5, cer_lit_rut5]
]
autocorrect = ["Without autocorrect",
               "SpellChecker",
               "HunSpell",
               "DeepPavlov",
               "JamSpell",
               "Aspell",
               "LanguageTool",
               "RU T5"]
pd.DataFrame(data=df, index=autocorrect,
             columns=["WER", "CER"])

Таблица сравнения автокорректоров для литературного текста


,WER,CER
Without autocorrect,0.046,0.004
SpellChecker,0.322,0.055
HunSpell,0.107,0.027
DeepPavlov,0.177,0.005
JamSpell,0.080,0.015
Aspell,0.108,0.081
LanguageTool,0.058,0.009
RU T5,0.491,0.314


In [ ]:
print("Таблица сравнения автокорректоров для учебного текста")
df_educ = [
    [0.023,	0.002],
  	[0.328,	0.047],
    [0.194,	0.016],
    [0.099,	0.003],
    [wer_educ_jamspell, cer_educ_jamspell],
    [wer_educ_aspell, cer_educ_aspell],
    [wer_educ_lt, cer_educ_lt],
    [wer_educ_rut5, cer_educ_rut5],
    [0.019, 0.002]

]
autocorrect = ["Without autocorrect",
               "SpellChecker",
               "HunSpell",
               "DeepPavlov",
               "JamSpell",
               "Aspell",
               "LanguageTool",
               "RU T5",
               "RuM2M100"]
pd.DataFrame(data=df_educ, index=autocorrect,
             columns=["WER", "CER"])

Таблица сравнения автокорректоров для учебного текста


,WER,CER
Without autocorrect,0.023,0.002
SpellChecker,0.328,0.047
HunSpell,0.194,0.016
DeepPavlov,0.099,0.003
JamSpell,0.028,0.003
Aspell,0.094,0.066
LanguageTool,0.053,0.005
RU T5,0.315,0.221
RuM2M100,0.019,0.002


**Выводы:**

Продолжила анализ автокорректоров. Загрузила и открыла распознанные файлы из первого задания. Протестировара новые автокорректоры, которые используют контекстную корректировку слов:

- **JamSpell**
- **Aspell**
- **LanguageTool**
- **RU T5** - предобученная модель на основе архитектуры T5 от Google
- **RuM2M100** - предобученная модель, основанная на архитектуре M2M100 от Meta

Почти все автокорректоры показали так же как и в предыдущей работе снижение метрик относительно распознанного текста без обработки автокорректором.

Кроме проблемы со словами, отсутствующими в словарях автокорректоров, появилась проблема неправильного понимания контекста моделями. Возможно, проблема в сложности текста, который я использовала. Литературный текст, который я использовала, написан писателями-фантастами, поэтому там встечаются фэнтезийные слова, жаргонизмы, разговорные слова. В тексте из учебника встречались сложносоставные слова (например, "правопонимание"), которые не всем корректорам удавалось распознать.

Только модель **RuM2M100** показала улучшение метрик относительно распознанного текста без обработки автокорректором.

На тексте из учебника до автокоррекции метрики были: WER 2,3%,	CER 0,2%

После автокоррекции **RuM2M100** метрики: WER	1,9%, CER 0,2%

Но у этой модели есть большой минус - скорость работы. Текст из учебника объемом 3 страницы обрабатывался 44 минуты. Исходя из этого, литературный тест был бы обработан за 4 часа. От этой идеи пришлось отказаться.

Таким образом, все автокорректоры ухудшают качество распознанного текста.